In [3]:
import keras
# Just checking the version 
# The time being it was 2.0.8

keras.__version__

'2.4.3'

In [4]:
from keras import layers, models

#### First Conv2D Network

In [5]:
model = models.Sequential()

# CONV Layer The input shaep is (HEiGHT, WIDTH , CHANNEL)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# Add a pooling layer with a max pool of 2x2 matrix
model.add(layers.MaxPooling2D((2, 2)))
# Another Conv Layer but this time with double the neuron and the matrix be the same 3 x3 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Again pooling  
model.add(layers.MaxPooling2D((2, 2)))
# Again conv layer 
model.add(layers.Conv2D(64, (3, 3), activation='relu'))



#### Architecture So Far

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________



You can see above that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as we go deeper in the network. The number of channels is controlled by the first argument passed to the Conv2D layers (e.g. 32 or 64).

The next step would be to feed our last output tensor (of shape (3, 3, 64)) into a densely-connected classifier network like those you are already familiar with: a stack of Dense layers. These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. So first, we will have to flatten our 3D outputs to 1D, and then add a few Dense layers on top:

In [7]:
model.add(layers.Flatten()) # converts the 3x3 shape to a single layer of 1D with 64 * 3 * 3 elements 
model.add(layers.Dense(64, activation='relu')) # Have a dense layer with RELU activation
model.add(layers.Dense(10, activation='softmax')) # Finally the SOFTMAX output layer (since we have more than 10 outputs for the mnist )

In [8]:
model.summary() # getting the summar yof the leemnt 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

#### Loading in the MNIST dataset

In [15]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.__len__(), test_images.__len__())
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
# Test images tensor matrix 
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
# Convert the output labels into categories 
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

60000 10000


In [16]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Fitting the model on the training images with epoch 5 and batch size be 64
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 31s 34ms/step - loss: 0.1770 - accuracy: 0.9445
Epoch 2/5
938/938 [==============================] - 31s 34ms/step - loss: 0.0494 - accuracy: 0.9854
Epoch 3/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0349 - accuracy: 0.9891
Epoch 4/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0266 - accuracy: 0.9920
Epoch 5/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0213 - accuracy: 0.9936


#### Model Evaluation

In [17]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.0262 - accuracy: 0.9922


#### Prediction

In [39]:
model.predict(np.array([test_images[0],]))

array([[4.32177840e-13, 3.03337133e-09, 8.74511663e-09, 3.77325051e-08,
        8.71103412e-10, 4.11995889e-11, 4.18358672e-18, 9.99999046e-01,
        1.12585505e-11, 9.70655719e-07]], dtype=float32)

#### Prediction all


In [41]:
model.predict(test_images)

array([[4.3217868e-13, 3.0333713e-09, 8.7451166e-09, ..., 9.9999905e-01,
        1.1258551e-11, 9.7065663e-07],
       [2.9959804e-06, 1.3184999e-04, 9.9986243e-01, ..., 1.8726858e-09,
        2.4493730e-07, 3.6599742e-12],
       [5.5853309e-09, 9.9999630e-01, 4.8317030e-08, ..., 1.8590669e-07,
        2.3769732e-07, 5.1841155e-08],
       ...,
       [1.8212408e-14, 2.6929527e-08, 3.5104605e-10, ..., 1.6383474e-08,
        3.8660207e-08, 2.1845425e-07],
       [5.1929476e-08, 8.0636764e-10, 2.2974627e-11, ..., 1.2103321e-10,
        5.7568112e-05, 5.9073713e-10],
       [3.2547894e-06, 7.8392182e-09, 5.0027396e-08, ..., 8.9766606e-12,
        5.8565570e-07, 2.1220612e-09]], dtype=float32)